##### Token Initialization

In [1]:
import pandas as pd
df=pd.read_csv('PoetryFoundationData.csv')
df.columns

col1=df['Poem']
entry=""
for i in range(0,21):
    for j in col1[i]:
        entry=entry+j

import re
text=re.split(r'(\s)', entry)
#Removing spaces

new_list=[]
for i in text:
    new=i.strip()
    if new!="":
        new_list.append(new)

#print(new_list)
new_list.sort()

##### Token ID

In [2]:
id={}
for i in range(len(new_list)):
    id.update({new_list[i]:i})

#print(id)

##### Tokenizer

In [3]:
class tokenizer:
    def __init__(self, dic):
        self.dic=dic

        dic1={}
        for i,j in dic.items():
            dic1.update({j:i})

        self.inverse_dic=dic1
    
    def encode(self, txt):
        text=re.split(r'([,:?;]|\s)', txt)
        id=[]
        for i in text:
            j=i.strip()
            if j!="":
                id.append(self.dic[j])
        
        return id
    
    def decode(self, id):
        st=""
        for i in range(len(id)):
            if id==len(id)-1:
                st=st+self.inverse_dic[id[i]]
            else:
                st=st+self.inverse_dic[id[i]]+" "
        
        return st

token1=tokenizer(id)
entry=token1.encode("It's a big Dog")
final=token1.decode(entry)
print(final)


It's a big Dog 


##### Byte Pair Encoding

In [4]:
import tiktoken
tokenizer1=tiktoken.get_encoding("gpt2")
codes=tokenizer1.encode("I love BMW")
text=tokenizer1.decode(codes)
print(text)

I love BMW


##### Sliding Window Algorithm

In [5]:
text_to_encode=col1[5]
limit=32
actual=text_to_encode[0:limit]

X=[]
Y=[]
for i in actual.split(" "):
    Y.append(i.strip())
    X.append(i.strip())


Y.remove(Y[0])
X.remove(X[-1])

print(actual+"\n")

X_id=[]
Y_id=[]
for i in X:
    X_id.append(id[i])

for i in Y:
    Y_id.append(id[i])

import torch

x_tensor=[]
y_tensor=[]

for i in range(0, len(X_id), 2):
    li=[]
    li.append(X_id[i])
    li.append(X_id[i+1])

    li1=[]
    li1.append(Y_id[i])
    li1.append(Y_id[i+1])

    x_tensor.append(li)
    y_tensor.append(li1)

x_tensor=torch.tensor(x_tensor)
y_tensor=torch.tensor(y_tensor)
print(x_tensor)
print(y_tensor)


You are the start of the week

tensor([[ 342,  582],
        [2694, 2396],
        [1956, 2694]])
tensor([[ 582, 2694],
        [2396, 1956],
        [2694, 2975]])


##### Embedding

In [6]:
embedding=torch.nn.Embedding(len(new_list), 3)
#print(embedding.weight)
embebbed_layer=embedding(x_tensor)
print(embebbed_layer)

tensor([[[ 2.3678,  0.2255, -0.8966],
         [ 0.0620, -0.1065,  0.2757]],

        [[ 0.8565,  0.2817, -0.9101],
         [ 0.1733, -0.2974, -1.3095]],

        [[-2.3439,  0.8081,  0.2367],
         [ 0.8565,  0.2817, -0.9101]]], grad_fn=<EmbeddingBackward0>)


##### Attention Scores

In [7]:
new_dim=[]
for i in embebbed_layer:
    for j in i:
        new_dim.append(j.tolist())   

new_dim=torch.tensor(new_dim)

attention=torch.empty(6,6)


for i,j in enumerate(new_dim):
    for k,l in enumerate(new_dim):
        attention[i,k]=torch.dot(j,l)

attention=torch.softmax(attention, dim=1)
print(attention)

context=attention@new_dim
print("\n--------------Context Vector---------------\n")
print(context)

tensor([[9.3829e-01, 1.2952e-03, 2.6858e-02, 6.6886e-03, 5.5341e-06, 2.6858e-02],
        [1.7163e-01, 2.1291e-01, 1.5476e-01, 1.4133e-01, 1.6461e-01, 1.5476e-01],
        [5.5354e-01, 2.4071e-02, 1.5604e-01, 1.0620e-01, 4.1101e-03, 1.5604e-01],
        [2.4061e-01, 3.8369e-02, 1.8536e-01, 3.3002e-01, 2.0272e-02, 1.8536e-01],
        [7.6103e-06, 1.7083e-03, 2.7424e-04, 7.7494e-04, 9.9696e-01, 2.7424e-04],
        [5.5354e-01, 2.4071e-02, 1.5604e-01, 1.0620e-01, 4.1101e-03, 1.5604e-01]])

--------------Context Vector---------------

tensor([[ 2.2689,  0.2246, -0.8986],
        [ 0.3234,  0.1942, -0.5230],
        [ 1.5882,  0.1819, -0.9118],
        [ 0.8993,  0.0728, -0.9699],
        [-2.3361,  0.8054,  0.2350],
        [ 1.5882,  0.1819, -0.9118]])


##### Attention Mechanism With Trainable Weights

In [8]:
# Input embebbed layer
input=embebbed_layer.reshape(-1, embebbed_layer.shape[2])


torch.manual_seed(150)
query=torch.nn.Parameter(torch.rand(3,2), requires_grad=False)
key=torch.nn.Parameter(torch.rand(3,2), requires_grad=False)
value=torch.nn.Parameter(torch.rand(3,2), requires_grad=False)

query_vector=input@query
key_vector=input@key
value_vector=input@value 

# Attention Score Calculation
attention_scores=query_vector @ key_vector.T

#Scaling and Normalization
key_shape=key_vector.shape[-1]
attention_scores_final=torch.softmax((attention_scores/(key_shape**0.5)), dim=1)
print("-----------ATTENTION SCORES-------------\n")
print(attention_scores_final)
#Context Vector
context_vector=attention_scores_final @ value_vector
print("-----------CONTEXT VECTOR-------------\n")
print(context_vector)


-----------ATTENTION SCORES-------------

tensor([[0.2937, 0.1581, 0.1730, 0.0697, 0.1326, 0.1730],
        [0.1796, 0.1684, 0.1682, 0.1454, 0.1702, 0.1682],
        [0.1863, 0.1646, 0.1709, 0.1571, 0.1504, 0.1709],
        [0.1041, 0.1525, 0.1475, 0.2888, 0.1595, 0.1475],
        [0.0870, 0.1421, 0.1380, 0.3501, 0.1448, 0.1380],
        [0.1863, 0.1646, 0.1709, 0.1571, 0.1504, 0.1709]],
       grad_fn=<SoftmaxBackward0>)
-----------CONTEXT VECTOR-------------

tensor([[0.7210, 0.7397],
        [0.4409, 0.4296],
        [0.4768, 0.4737],
        [0.2474, 0.2297],
        [0.2036, 0.1891],
        [0.4768, 0.4737]], grad_fn=<MmBackward0>)


##### Causal Attention Mechanism

In [21]:
# Using Previous Attention Vector

masked_matrix=torch.tril(torch.ones(6,6))
new_attention=attention_scores_final * masked_matrix
print("-----------ATTENTION SCORES-------------\n")

#Normalization
row=new_attention.sum(dim=1, keepdim=True)
masked_attention=new_attention/row

#Fixing Normalization Problem
fix=torch.triu(torch.ones(6,6), diagonal=1)
fixed_attention=attention_scores_final.masked_fill(fix.bool(), -torch.inf)

#Softmax with scaling
attention_score=torch.softmax(fixed_attention/(key_shape**0.5), dim=1)

#Dropout Phase
dropout=torch.nn.Dropout(0.4)
attention_score_1=dropout(attention_score)

print(attention_score_1)
print("-----------CONTEXT VECTOR-------------\n")
context_vector_causal=attention_score_1 @ value_vector
print(context_vector_causal)



-----------ATTENTION SCORES-------------

tensor([[1.6667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8366, 0.8300, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5604, 0.0000, 0.5544, 0.0000, 0.0000, 0.0000],
        [0.3963, 0.0000, 0.4087, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3246, 0.0000, 0.0000, 0.0000],
        [0.2816, 0.0000, 0.2786, 0.2759, 0.2746, 0.0000]],
       grad_fn=<MulBackward0>)
-----------CONTEXT VECTOR-------------

tensor([[3.2372, 3.5236],
        [1.5871, 1.7543],
        [1.5730, 1.6797],
        [1.1269, 1.2026],
        [0.2837, 0.2898],
        [0.4707, 0.4397]], grad_fn=<MmBackward0>)
